In [95]:
# https://stackoverflow.com/questions/33289820/noun-phrases-with-spacy

In [4]:
from reader import *

In [3]:
from fuzzywuzzy import fuzz

In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [5]:
input_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/a9.input'
key_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/a9.key'

In [6]:
reader = ReadInput(input_path)
list_of_sentences = reader.getListOfSentences()
# print(list_of_sentences)
full_text = reader.getText()
# print(full_text)

In [7]:
sentence_dict = getSentenceDict(list_of_sentences)
cluster_head_dict = getClusterHeads(sentence_dict)

In [8]:
def getRemText(sent_id, sentence_dict):
    rem_text = {}
    for sid in sentence_dict.keys():
        if(sid >= sent_id): 
            rem_text[sid] = sentence_dict[sid]
    sorted_rem_text = dict(sorted(rem_text.items(), key = lambda x:x[0]))
    return sorted_rem_text

In [9]:
def getSimilarityScore(cluster_head_name,noun_phrase):
    ## When NP will be used, then partial_ratio will be used
    return fuzz.partial_ratio(cluster_head_name.lower(),noun_phrase.lower_)

In [1]:
def getCorefDict_match_NP(sentence_dict, cluster_head_dict, threshold):
    
    ## Initialise a coref_dict 
    coref_dict = {}
    
    for cluster_id, cluster_value in cluster_head_dict.items():

        current_cluster_sent_id = cluster_value[0]        
        current_cluster_head = cluster_value[1]
        
        remaining_text = getRemText(current_cluster_sent_id, sentence_dict)
        
        for sid, sentence in remaining_text.items():

            clean = re.compile('<COREF .*?>.*?</COREF>') ## removing all coref tags from the current sentence
            clean_sentence = re.sub(clean, '', sentence)

            doc = nlp(clean_sentence)
            
            for np in doc.noun_chunks:
                
                ## If the NP is a pronoun, ignore it. 
                ## We are handling pronouns by Hobb's algorithm
                if(len(np) == 1 and np[0].pos_ == 'PRON'):
                    continue
                
                similarity_score = getSimilarityScore(current_cluster_head,np)
                
                if(similarity_score > threshold):
                    if(cluster_id not in coref_dict.keys()):

                        coref_dict[cluster_id] = [list([current_cluster_head, sid, np.text, similarity_score])]
                    else:
                        coref_dict[cluster_id].append(list([current_cluster_head, sid, np.text, similarity_score]))
    return coref_dict

In [11]:
coref_dict = getCorefDict_match_NP(sentence_dict, cluster_head_dict,80)

In [12]:
coref_dict

{'X0': [['F14-CRASHES', 5, BC-F14-CRASHES-BLOOM, 100],
  ['F14-CRASHES', 10, three recent F-14 crashes, 82]],
 'X1': [['02-22', 34, NYT-02-22-96 1221EST, 100]],
 'X2': [['NORTHROP GRUMMAN F-14S', 13, an F-14D, 88],
  ['NORTHROP GRUMMAN F-14S', 14, an F-14A, 88],
  ['NORTHROP GRUMMAN F-14S', 17, all F-14s, 82],
  ['NORTHROP GRUMMAN F-14S', 29, Northrop Grumman, 100],
  ['NORTHROP GRUMMAN F-14S', 33, Northrop Grumman shares, 82]],
 'X4': [['Navy', 12, the Navy, 100],
  ['Navy', 16, the Navy, 100],
  ['Navy', 17, the Navy, 100],
  ['Navy', 20, the Navy's first-line fighter aircraft, 100],
  ['Navy', 22, The Navy, 100],
  ['Navy', 26, The Navy, 100],
  ['Navy', 27, Navy jet fighter aircraft, 100],
  ['Navy', 30, the Navy, 100],
  ['Navy', 31, the Navy, 100]],
 'X5': [['three days', 32, three days, 100]],
 'X6': [['Persian Gulf', 18, the Persian Gulf, 100]],
 'X7': [['a 72-hour safety standdown', 18, the standdown, 88]],
 'X8': [['Lt. Greg Geisen', 18, Geisen, 100],
  ['Lt. Greg Geisen', 27

In [13]:
reader_o = ReadInput(key_path)
ans = reader_o.getListOfSentences()

In [14]:
ans

['<COREF ID="X0">F14-CRASHES</COREF>\n',
 '{5} {F14-CRASHES} {CRASHES}\n',
 '{6} {ACCIDENTS} {ACCIDENTS}\n',
 '{10} {three recent F-14 crashes} {crashes}\n',
 '{16} {the recent mishaps involving F-14As and Ds} {mishaps}\n',
 '{17} {these mishaps} {mishaps}\n',
 '{32} {the three recent crashes} {crashes}\n',
 '\n',
 '<COREF ID="X1">02-22</COREF>\n',
 '{10} {Feb. 22} {Feb. 22}\n',
 '{11} {today} {today}\n',
 '{34} {02-22-96} {02-22-96}\n',
 '\n',
 '<COREF ID="X2">NORTHROP GRUMMAN F-14S</COREF>\n',
 '{10} {its Northrop Grumman Corp. F-14s} {F-14s}\n',
 '{16} {all F-14 aircraft} {F-14 aircraft}\n',
 '{17} {all F-14s} {F-14s}\n',
 '{22} {about 330 F-14s} {F-14s}\n',
 '{29} {The planes} {planes}\n',
 '\n',
 '<COREF ID="X3">Bloomberg Business News</COREF>\n',
 '{10} {Bloomberg} {Bloomberg}\n',
 '\n',
 '<COREF ID="X4">Navy</COREF>\n',
 '{10} {it} {it}\n',
 '{12} {Navy} {Navy}\n',
 '{16} {Navy} {Navy}\n',
 '{17} {Navy} {Navy}\n',
 '{22} {Navy} {Navy}\n',
 '{26} {Navy} {Navy}\n',
 '{27} {Navy} {